In [ ]:
import pandas as pd
import numpy as np
import re

#**Step 0: Install packages and set up OpenAI**

In [ ]:
!pip install openai==0.28

In [ ]:
import openai

In [ ]:
openai.api_key = 'insert-your-openai-key'

#**Step 1: Named Entity Extraction Code - detect location and pro-surfer names**

In [ ]:
import tqdm

In [ ]:
def extract_info(texts):
    results = []
    for text in tqdm.tqdm(texts):
        # Construct the system and user messages for the chat completion
        messages = [
            {"role": "system", "content": "Extract the geographical locations and celebrity names from the following text."},
            {"role": "user", "content": text}
        ]
        # Make an API call for each text using the chat completions endpoint
        completion = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=150
        )
        # Collect the response
        extracted_info = completion['choices'][0]['message']['content'] if completion['choices'] else 'No output generated'
        results.append(extracted_info)
        #rint(f"Processed: {text} -> Extracted: {extracted_info}")
    return results

#**Step 2: Load the cleaned data which was earlier scrapped from YouTube**

In [ ]:
data = pd.read_csv('/content/video_all.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data

,video_id,channelTitle,title,description,tags,publishedAt,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption,month_year,day,month,video_length
0,DNthlDYvhzM,Surf Splendor - Surf Podcast Network,Skeleton Bay Righthander Discovered & History ...,"Trans Surfers, Skeleton Bay Right Discovered, ...","['beach podcast', 'best surfing podcast', 'cal...",2024-05-16T01:18:29Z,{'default': {'url': 'https://i.ytimg.com/vi/DN...,905.0,21.0,NaN,10.0,1:10:17,hd,False,2024-05,Thursday,5,70.283333
1,xUw8b2EtB30,Surf Splendor - Surf Podcast Network,"Malibu's Most Dangerous Waves, Laird Get's Fle...",Secret to Standing Your Ground in the Lineup 🥊...,"['beach podcast', 'beachgrit', 'beachgrit news...",2024-05-13T21:30:06Z,{'default': {'url': 'https://i.ytimg.com/vi/xU...,359.0,3.0,NaN,4.0,9:4,hd,False,2024-05,Monday,5,9.066667
2,INdN6BdKl4c,Surf Splendor - Surf Podcast Network,"From Winks to Feuds: Drake vs. Kendrick, and M...","The Art of the Wink, Audiobooks and Road Trips...","['beach podcast', 'beachgrit', 'beachgrit news...",2024-05-13T20:45:00Z,{'default': {'url': 'https://i.ytimg.com/vi/IN...,93.0,2.0,NaN,0.0,17:5,hd,False,2024-05,Monday,5,17.083333
3,BVq5KEcPg-Y,Surf Splendor - Surf Podcast Network,"Epic Surfing Rule Breakers, F🖕You Laird, and T...",Latest surf news - Pushing Through Pain for Su...,"['beach podcast', 'beachgrit news', 'beachgrit...",2024-05-11T21:00:04Z,{'default': {'url': 'https://i.ytimg.com/vi/BV...,1610.0,24.0,NaN,31.0,1:16:43,hd,False,2024-05,Saturday,5,76.716667
4,XZZHO1t8dh8,Surf Splendor - Surf Podcast Network,The Story of Sam Hammer | Navigating Challenge...,Sam Hammer's Surfing Journey - New Jersey Nati...,"['beach podcast', 'best surf podcast', 'histor...",2024-05-09T21:00:25Z,{'default': {'url': 'https://i.ytimg.com/vi/XZ...,1067.0,22.0,NaN,4.0,1:17:1,hd,False,2024-05,Thursday,5,77.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39282,2Jq4Ryij1yQ,Nate & Koa Podcast,#5 BEST BOARD FOR AMATUR SURFERS!? PYZELS NEW ...,Episode 5 of the Nate and Koa podcast. And for...,NaN,2023-02-03T18:53:25Z,{'default': {'url': 'https://i.ytimg.com/vi/2J...,91173.0,2638.0,NaN,281.0,50:43,hd,False,2023-02,Friday,2,50.716667
39283,jKQdnkd_mD4,Nate & Koa Podcast,#4 EDDIE CROWD DESTROYED BY SURGE WAVE NATES S...,Ep.4 of the Nate and koa podcast! \nSubscribe ...,NaN,2023-01-26T23:23:45Z,{'default': {'url': 'https://i.ytimg.com/vi/jK...,86314.0,3723.0,NaN,527.0,39:20,hd,False,2023-01,Thursday,1,39.333333
39284,q56xMSQVoSk,Nate & Koa Podcast,"#3 MOST DANGEROUS SECTION AT PIPE, RETURNING T...",Ep. 3! Hope you guys are enjoying the podcast!...,NaN,2023-01-19T21:09:07Z,{'default': {'url': 'https://i.ytimg.com/vi/q5...,71166.0,2423.0,NaN,480.0,54:10,hd,False,2023-01,Thursday,1,54.166667
39285,de8sppV2Bmk,Nate & Koa Podcast,"#2 JAWS VS WAIMEA, SPINAL INJURIES, TRADING ST...",@NathanFlorence @KoaRothman talk about big wa...,NaN,2023-01-14T03:58:01Z,{'default': {'url': 'https://i.ytimg.com/vi/de...,82078.0,3621.0,NaN,707.0,44:14,hd,False,2023-01,Saturday,1,44.233333


In [ ]:
titles = list(data['title'])
titles

["Skeleton Bay Righthander Discovered & History Made by Caitlin's Barrel Clinic!",
 "Malibu's Most Dangerous Waves, Laird Get's Flexed On, and Manhood Tips",
 "From Winks to Feuds: Drake vs. Kendrick, and Mother's Day Gifts!",
 'Epic Surfing Rule Breakers, F🖕You Laird, and The SECRET to Aging Like a Boss!',
 'The Story of Sam Hammer | Navigating Challenges and Finding Balance in Surfing',
 'Surfers Shot and Buried in Baja, Surfer Magazine Returns, and WSL Controversy',
 'The Art of Lifelong Surfing, Embracing Criticism, Kookiest Content Creators',
 'Religious Cussers, The Epic Leash Pull, and Surf Attire in Public: "Barrel or Nah?"',
 'WSL Apologies, Fish Dick, No Gay Pros, Jesus, and The Joy of a Leash Pull!',
 'How Cheyne Horan Changed the Surfing Game Forever! #surfinglegends',
 "🤼\u200d♂️Surf Parties and Bar Fights, Kelly's Hips Return, Olympic GOAT? 🐐",
 'Partner or Pal? Foot Wax, Ketchup vs. Salsa, and Ted Lasso!',
 '👋 New Rivalry Brewing: Devon Slaps David and Chas |  Mid-Length

For illustration purposes, the code will be run on 200 titles.

In [ ]:
extracted_data = extract_info(titles[:1000]) #extracted_data = extract_info(titles) to run the entire data set
print(extracted_data)

100%|██████████| 1000/1000 [11:10<00:00,  1.49it/s]

['**Geographical Location:** Skeleton Bay\n\n**Celebrity Name:** Caitlin', 'Geographical Location:\n- Malibu\n\nCelebrity Name:\n- Laird', '**Geographical Locations:**\n\n- No specific geographical locations mentioned. \n\n**Celebrity Names:**\n\n- Drake\n- Kendrick', 'Geographical locations: None\nCelebrity names: Laird', 'Without the provided text, I cannot extract specific geographical locations and celebrity names. Could you please provide the text content for your request?', 'Geographical Locations:\n- Baja\n\nCelebrity Names:\n- None', 'The text does not provide any geographical locations or celebrity names.', 'The text does not contain any geographical locations or celebrity names.', 'There are no geographical locations or celebrity names mentioned in the text provided.', 'Geographical locations: None\n\nCelebrity names: Cheyne Horan', 'The text does not explicitly mention any geographical locations. It mentions the celebrity name "Kelly\'s Hips," which might refer to surfer Kel

#**Step 3: Clean output from OpenAI so it is usable for analysis**

As you can see from the earlier output from OpenAI, the data *loosely* follows a structure, here we are cleaning up the output to get the locations neatly extracted and to also address things like misspelt names and locations.

In [ ]:
def clean_text(text_4_cleaning):
  cleaned_text = []

  indicators = ['not specified', 'not explictly mentioned', 'nothing mentioned', 'not mentioned', 'not applicable', 'none mentioned', 'not provided', 'not provided in text', 'n/a', 'none' ]

  for text in text_4_cleaning:
    text = str(text)
    cleaned = text.lower().strip().replace('-','').replace('.','').replace('\n','').replace('"', '')

    for indicator in indicators:
      cleaned = cleaned.replace(indicator, 'nothing')

    cleaned_text.append(cleaned)

  return cleaned_text

In [ ]:
cleaned_extracted_data = clean_text(extracted_data)

In [ ]:
cleaned_extracted_data

['**geographical location:** skeleton bay**celebrity name:** caitlin',
 'geographical location: malibucelebrity name: laird',
 '**geographical locations:** no specific geographical locations mentioned **celebrity names:** drake kendrick',
 'geographical locations: nothingcelebrity names: laird',
 'without the provided text, i cannot extract specific geographical locations and celebrity names could you please provide the text content for your request?',
 'geographical locations: bajacelebrity names: nothing',
 'the text does not provide any geographical locations or celebrity names',
 'the text does not contain any geographical locations or celebrity names',
 'there are no geographical locations or celebrity names mentioned in the text provided',
 'geographical locations: nothingcelebrity names: cheyne horan',
 "the text does not explicitly mention any geographical locations it mentions the celebrity name kelly's hips, which might refer to surfer kelly slater, but it is not clearly stat

In [ ]:
def find_colons(text):
    return text.count(':')

def split_text(texts_for_splitting):
    locations = []
    surfers = []

    geo_indicators = ['geographical location: ', 'geographical locations: ']
    celeb_indicators = ['celebrity names: ', 'celebrities: ', 'celebrity: ', 'celebrity name: ']

    for text in texts_for_splitting:
        num_colons = find_colons(text)

        if num_colons == 0 or 'no * geographical locations or celebrity names mentioned' in text:
            locations.append('nothing')
            surfers.append('nothing')
        else:
            location_part = None
            celebrity_part = None

            # Check for geographical location indicators
            for indicator in geo_indicators:
                if indicator in text:
                    split_text = text.split(indicator, 1)
                    location_part = split_text[1].split(':')[0] if len(split_text) > 1 else 'nothing'
                    break

            # Check for celebrity indicators
            for indicator in celeb_indicators:
                if indicator in text:
                    if location_part:
                        celebrity_part = text.split(indicator, 1)[1]
                    else:
                        split_text = text.split(indicator, 1)
                        celebrity_part = split_text[1] if len(split_text) > 1 else 'nothing'
                    break

            locations.append(location_part if location_part else 'nothing')
            surfers.append(celebrity_part if celebrity_part else 'nothing')

    return locations, surfers

In [ ]:
loc, surfers = split_text(cleaned_extracted_data)

In [ ]:
loc

['nothing',
 'malibucelebrity name',
 'nothing',
 'nothingcelebrity names',
 'nothing',
 'bajacelebrity names',
 'nothing',
 'nothing',
 'nothing',
 'nothingcelebrity names',
 'nothing',
 'nothing',
 'nothingcelebrity names',
 'nothing',
 'margaret rivercelebrity names',
 'maui  celebrity name',
 "marg's (presumably a reference to margaret river, australia)celebrity name",
 'nothingcelebrity names',
 'nothing',
 'nothing',
 'nothingcelebrity names',
 'nothingcelebrity names',
 'nothingcelebrity names',
 'amazoncelebrity name',
 'nothing',
 'nothingcelebrity name',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing explicitlycelebrity name',
 'nothing',
 'nothing specifiedcelebrity name',
 'nothing',
 'nothing',
 'nothing in the text providedcelebrity names',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'japancelebrity name',
 'jawscelebrity name',
 'nothing',
 'nothing',
 'nothingcelebrity names',
 'nothing',
 'nothingcelebrity names',
 'nothing',
 'nothingcelebrity names',


In [ ]:
surfers

['nothing',
 'laird',
 'nothing',
 'laird',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'cheyne horan',
 'nothing',
 'nothing',
 'devon, david, chas',
 'nothing',
 'nothing in the text',
 'timpone',
 'slater (likely referring to kelly slater, a famous professional surfer)',
 'nic timpone',
 'nothing',
 'nothing',
 'kelly slater',
 'kelly slater',
 'tony moniz',
 'kelly slater',
 'nothing',
 'shane stoneman',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'shane stoneman',
 'nothing',
 'gabriel medina',
 'nothing',
 'nothing',
 'slater (presumably referring to kelly slater)',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'ajw',
 'adam warden',
 'nothing',
 'nothing',
 'kelly slater, tony robbins',
 'nothing',
 'kelly slater, dane reynolds',
 'nothing',
 'moore dior',
 'nothing',
 'nothing',
 'adam',
 'nothing',
 'derek hynd',
 'nothing',
 'nothing',
 'kelly slater gerry lopez',
 'clint runyon',
 'nothing',
 'alex “superwolf” villalobos',
 'alex “superwolf” villalob

#**Step 3a: Extract the locations**

In [ ]:
def process_locations(loc):
    error_indicators = ['there are no ', 'there is no', 'nothing in the', 'nothing found', 'mentioned', 'not found', 'unknown', 'not available', 'does not', 'mention', 'not present', 'nothing']
    # Create a regex pattern that is case-insensitive
    error_pattern = re.compile('|'.join([re.escape(ind) for ind in error_indicators]), re.IGNORECASE)
    collection_list = []

    for i in loc:
        # Clean and prepare the test
        i1 = i.split('celebrity')[0]
        i2 = i1.replace('celebrities', '').strip()

        # Use regex to search for any error indicator
        if error_pattern.search(i2) or i2=='no':
          i2 = 'nothing'
          collection_list.append(i2)
        else:
          collection_list.append(i2)

    return collection_list

In [ ]:
loc_1 = process_locations(loc)
loc_1

['nothing',
 'malibu',
 'nothing',
 'nothing',
 'nothing',
 'baja',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'margaret river',
 'maui',
 "marg's (presumably a reference to margaret river, australia)",
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'amazon',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'japan',
 'jaws',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'clark foam',
 'nothing',
 'cloudbreak',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'baja',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'brazilian, sunset',
 'nothing',
 'north island',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',

#**Step 3b: Extract the pro-surfers**

In [ ]:
def refine_pros(pro):
    pro_list = []

    # Mapping of common substrings to standardized pro surfer names. Note that this can be adapted for locations as well.
    pro_mapping = {
        'there are no': 'nothing',
        'nothing': 'nothing',
        'no specific': 'nothing',
        'gromsearch': 'nothing',
        'cup': 'nothing',
        'rip curl': 'nothing',
        'mick': 'mick fanning',
        'fanning': 'mick fanning',
        'gabriel medina': 'gabriel medina',
        'medina': 'gabriel medina',
        'owen wright': 'owen wright',
        'mason ho': 'mason ho',
        'tyler wright': 'tyler wright',
        'dillon perillo': 'dillon perillo',
        'matt wilkinson': 'matt wilkinson',
        'tom curren': 'tom curren',
        'bethany hamilton': 'bethany hamilton',
        'alana blanchard': 'alana blanchard',
        'chris rasman': 'chris rasman',
        'conner coffin': 'conner coffin',
        'crosby colapinto':'crosby colapinto',
        'slater': 'kelly slater',
        'garut widiarta': 'garut widiarta',
        'florence': 'john john florence',
        'taylor knox': 'taylor knox',
        'heather brown': 'heather brown',
        'molly picklum': 'molly picklum',
        'jordy smith': 'jordy smith',
        'taj burrow': 'taj burrow'
    }

    for item in pro:
        matched = False
        for key, value in pro_mapping.items():
            if key in item:
                pro_list.append(value)
                matched = True
                break
        if not matched:
            pro_list.append(item)  # Append the original item if no match found

    return pro_list


In [ ]:
pro_c1 = refine_pros(surfers)
pro_c1

['nothing',
 'laird',
 'nothing',
 'laird',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'cheyne horan',
 'nothing',
 'nothing',
 'devon, david, chas',
 'nothing',
 'nothing',
 'timpone',
 'kelly slater',
 'nic timpone',
 'nothing',
 'nothing',
 'kelly slater',
 'kelly slater',
 'tony moniz',
 'kelly slater',
 'nothing',
 'shane stoneman',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'shane stoneman',
 'nothing',
 'gabriel medina',
 'nothing',
 'nothing',
 'kelly slater',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'ajw',
 'adam warden',
 'nothing',
 'nothing',
 'kelly slater',
 'nothing',
 'kelly slater',
 'nothing',
 'moore dior',
 'nothing',
 'nothing',
 'adam',
 'nothing',
 'derek hynd',
 'nothing',
 'nothing',
 'kelly slater',
 'clint runyon',
 'nothing',
 'alex “superwolf” villalobos',
 'alex “superwolf” villalobos',
 'nothing',
 'john peck',
 'felipe toledo',
 'gerry lopez',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'nothing',
 'scott',
 'nothing'

#**Step 4: Integrating the cleaned text into the rest of the dataset**

Done on the top 1000 rows for demonstration purposes. Pleasee note that more thorough cleaning can be done when executed on specific channels.

In [ ]:
data_trunc_df = data[:1000]
data_trunc_df

,video_id,channelTitle,title,description,tags,publishedAt,thumbnails,viewCount,likeCount,dislikeCount,commentCount,duration,definition,caption,month_year,day,month,video_length
0,DNthlDYvhzM,Surf Splendor - Surf Podcast Network,Skeleton Bay Righthander Discovered & History ...,"Trans Surfers, Skeleton Bay Right Discovered, ...","['beach podcast', 'best surfing podcast', 'cal...",2024-05-16T01:18:29Z,{'default': {'url': 'https://i.ytimg.com/vi/DN...,905.0,21.0,NaN,10.0,1:10:17,hd,False,2024-05,Thursday,5,70.283333
1,xUw8b2EtB30,Surf Splendor - Surf Podcast Network,"Malibu's Most Dangerous Waves, Laird Get's Fle...",Secret to Standing Your Ground in the Lineup 🥊...,"['beach podcast', 'beachgrit', 'beachgrit news...",2024-05-13T21:30:06Z,{'default': {'url': 'https://i.ytimg.com/vi/xU...,359.0,3.0,NaN,4.0,9:4,hd,False,2024-05,Monday,5,9.066667
2,INdN6BdKl4c,Surf Splendor - Surf Podcast Network,"From Winks to Feuds: Drake vs. Kendrick, and M...","The Art of the Wink, Audiobooks and Road Trips...","['beach podcast', 'beachgrit', 'beachgrit news...",2024-05-13T20:45:00Z,{'default': {'url': 'https://i.ytimg.com/vi/IN...,93.0,2.0,NaN,0.0,17:5,hd,False,2024-05,Monday,5,17.083333
3,BVq5KEcPg-Y,Surf Splendor - Surf Podcast Network,"Epic Surfing Rule Breakers, F🖕You Laird, and T...",Latest surf news - Pushing Through Pain for Su...,"['beach podcast', 'beachgrit news', 'beachgrit...",2024-05-11T21:00:04Z,{'default': {'url': 'https://i.ytimg.com/vi/BV...,1610.0,24.0,NaN,31.0,1:16:43,hd,False,2024-05,Saturday,5,76.716667
4,XZZHO1t8dh8,Surf Splendor - Surf Podcast Network,The Story of Sam Hammer | Navigating Challenge...,Sam Hammer's Surfing Journey - New Jersey Nati...,"['beach podcast', 'best surf podcast', 'histor...",2024-05-09T21:00:25Z,{'default': {'url': 'https://i.ytimg.com/vi/XZ...,1067.0,22.0,NaN,4.0,1:17:1,hd,False,2024-05,Thursday,5,77.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,u7FXYULl764,Italo Ferreira,"COMO ENTREI NA AUSTRÁLIA, FOI DIFÍCIL - ITALO ...","#Austrália\nFinalmente os eventos voltaram, ma...","['Italo ferreira', 'italo', 'ferreira surf', '...",2021-04-06T21:15:16Z,{'default': {'url': 'https://i.ytimg.com/vi/u7...,70562.0,4039.0,NaN,250.0,10:47,hd,True,2021-04,Tuesday,4,10.783333
996,xksuVpLSW9k,Italo Ferreira,COMO NÃO JOGAR GOLF - ITALO FERREIRA#19,"A viagem do Italo Ferreira continua, depois de...","['italo ferreira', 'redbull', 'california', 'g...",2021-03-30T21:03:09Z,{'default': {'url': 'https://i.ytimg.com/vi/xk...,41060.0,2377.0,NaN,119.0,11:17,hd,True,2021-03,Tuesday,3,11.283333
997,0aA292G_SZQ,Italo Ferreira,FECHEI COM A MAIOR EMPRESA DO MUNDO - REDBULL #18,Sempre foi um sonho poder entrar para a equipe...,"['italo ferreira', 'italo', 'italo surfing', '...",2021-03-22T07:15:16Z,{'default': {'url': 'https://i.ytimg.com/vi/0a...,152178.0,6713.0,NaN,317.0,12:42,hd,True,2021-03,Monday,3,12.700000
998,TzL9pUcKYyk,Italo Ferreira,ITALO FERREIRA DINÂMICO - FERNANDO DE NORONHA,Selecionamos as melhores ondas e o resultado é...,"['italo ferreira', 'italo surf', 'italo campea...",2021-03-15T21:00:16Z,{'default': {'url': 'https://i.ytimg.com/vi/Tz...,54512.0,2543.0,NaN,149.0,2:55,hd,False,2021-03,Monday,3,2.916667


In [ ]:
data_trunc_df['pros'].value_counts()

,count
pros,
nothing,696
italo ferreira,26
john john florence,21
kelly slater,16
john john,8
...,...
jason baffa,1
terrence martin,1
lane davey,1


In [ ]:
data_trunc_df['locations'].value_counts()

,count
locations,
nothing,868
portugal,7
california,5
surf ranch,4
baja,3
...,...
pipe,1
the united states,1
florence,1


In [ ]:
data_trunc_df['locations'] = loc_1

<ipython-input-24-4537727b0981>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_trunc_df['locations'] = loc_1


In [ ]:
data_trunc_df['pros'] = pro_c1

<ipython-input-25-9339bf0a5d3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_trunc_df['pros'] = pro_c1


In [ ]:
data_trunc_df.to_csv('data_trunc_extracted_pros_n_locations.csv')